# --- Day 9: Disk Fragmenter ---

https://adventofcode.com/2024/day/9

## Parse the Input Data

In [1]:
def parse(filename):
    """Parse input data for puzzle.

    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.

    Returns
    -------
    disk_map : str
    """
    with open(f'../inputs/{filename}.txt') as f:
        disk_map = [int(c) for c in f.read().strip()]
        return disk_map

## Part 1
---

In [2]:
from collections import deque

In [3]:
def sum_seq(start, stop):
    assert start <= stop
    n = stop - start + 1
    return n * start + ((n * (n - 1)) / 2)

In [4]:
def solve(dm):
    """ dm == 'disk map' """
    checksum = 0
    # Every odd position in dm corresponds to an ID value
    ids = list(range(int((len(dm) + 1) / 2)))
    i = 0  # Use to keep track of odd/even location in dm
    pos = 0  # Use to keep track of location of final file layout
    dq = deque()  # Use to hold IDs from the tail to slot into spaces

    while dm:
        n = int(dm.pop(0))

        if not i % 2:
            # Odd positions (0, 2, 4, etc.) corresponding to files
            checksum += sum_seq(pos, pos + n - 1) * ids.pop(0)
            pos += n

        elif n > 0:
            # Even positions (1, 3, 5, etc.) corresponding to spaces
            while len(dq) <= n:
                dq.extend([ids.pop()] * dm.pop())
                dm.pop()  # Don't need the spaces on the end...

            for p in range(pos, pos + n):
                checksum += p * dq.popleft()

            pos += n

        i += 1

    for _ in range(len(dq)):
        checksum += pos * dq.popleft()
        pos += 1

    return int(checksum)

In [5]:
solve([1, 2, 3, 4, 5]) == 60

True

### Run on Test Data

In [6]:
solve(parse('test_disk_map')) == 1928

True

### Run on Input Data

In [7]:
solve(parse('disk_map'))

6258319840548

## Part 2
---

In [8]:
def expand(dm):
    block_map = ''
    id_counter = 0
    pos_counter = 0
    pos_dict = {}
    ids_dict = {}

    for i, n in enumerate(dm):
        if not i % 2:
            ids_dict[id_counter] = {}
            ids_dict[id_counter]['pos'] = pos_counter
            ids_dict[id_counter]['len'] = int(n)

            block_map += 'f' * int(n)

            for _ in range(int(n)):
                pos_dict[pos_counter] = id_counter
                pos_counter += 1

            id_counter += 1

        elif int(n) > 0:
            block_map += '.' * int(n)

            for _ in range(int(n)):
                pos_dict[pos_counter] = 0
                pos_counter += 1

    return block_map, pos_dict, ids_dict

In [9]:
import numpy as np

In [10]:
def solve2(dm):
    block_map, pos_dict, ids_dict = expand(dm)
    checksum = 0

    for f in list(ids_dict.keys())[::-1]:
        file_len = ids_dict[f]['len']
        file_pos = ids_dict[f]['pos']
        space_pos = block_map.find('.' * file_len)

        if space_pos != -1 and space_pos < file_pos:
            # Update block_map
            block_map = block_map[:space_pos] + ('x' * file_len) + block_map[space_pos + file_len:]

            # Move file
            for p in range(space_pos, space_pos + file_len):
                pos_dict[p] = f
            for p in range(file_pos, file_pos + file_len):
                pos_dict[p] = 0

            ids_dict[f]['pos'] = space_pos

    checksum = np.dot(np.array(list(pos_dict.keys())), np.array(list(pos_dict.values())))

    return checksum

### Run on Test Data

In [11]:
solve2(parse('test_disk_map')) == 2858

True

### Run on Input Data

In [12]:
solve2(parse('disk_map'))

6286182965311